In [ ]:
# 🕒 Timer global
global_start = time.time()

# 📥 Étape 1 : Chargement des données
start = time.time()

parcelles = gpd.read_file("parcelles_filtrées.geojson")
print(f"📦 Nombre total de parcelles chargées : {len(parcelles)}")


print("📥 Chargement des fichiers...")

print(f"✅ Données chargées en {time.time() - start:.2f} sec")
# 🏘️ Étape 6 : Récupération des zones d'urbanisme
start = time.time()
print("🏘️ Attribution des zones d’urbanisme ('typezone')...")

zones_urba = gpd.read_file("zones_urbanisme_sur_parcelles.geojson")

# 🗺️ Harmonisation des projections
zones_urba = zones_urba.to_crs(parcelles.crs)

# 📍 Spatial join entre parcelles et zones d'urbanisme
parcelles_zones = gpd.sjoin(
    parcelles,
    zones_urba[['geometry', 'typezone']],
    how="left",
    predicate="intersects"
)

# 🧼 Regrouper les zones par parcelle
parcelles_zones_grouped = (
    parcelles_zones
    .groupby(['commune', 'section', 'numero', 'prefixe', 'Code site'], dropna=False)
    .agg({
        'geometry': 'first',  # garder la géométrie d'une des lignes
        'typezone': lambda x: ', '.join(sorted(set(filter(pd.notnull, x))))
    })
    .reset_index()
)

# 🧭 Assurer le CRS si nécessaire
parcelles_zones_grouped = gpd.GeoDataFrame(parcelles_zones_grouped, geometry='geometry', crs=parcelles.crs)


# ✅ Renommage correct
parcelles_zones_grouped.rename(columns={"typezone": "zone_urbanisme"}, inplace=True)

# 💾 Export
print("💾 Export vers 'parcelles_analyse_zonage.geojson' et Excel...")

parcelles_zones_grouped.to_file("parcelles_analyse_zonage_Urbanisme.geojson", driver="GeoJSON")
parcelles_zones_grouped[[
    'Code site', 'commune', 'section', 'numero', 'prefixe', 'zone_urbanisme'
]].to_excel("parcelles_analyse_zonage_Urbanisme.xlsx", index=False)

print(f"✅ Export terminé en {time.time() - start:.2f} sec")
print(f"🎉 Script complet exécuté en {time.time() - global_start:.2f} sec")
